# 安裝套件與確認 SDK 版本

In [ ]:
import os
import azureml.core
from azureml.core import (
    Workspace,
    Experiment,
    Dataset,
    Datastore,
    ComputeTarget,
    Environment,
    ScriptRunConfig
)
from azureml.data import OutputFileDatasetConfig
from azureml.core.compute import AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.pipeline.core import PublishedPipeline
from azureml.pipeline.core import PipelineData

print(f"Azure ML SDK Version: {azureml.core.VERSION}")

# 配置工作環境

In [ ]:
workspace = Workspace.from_config()
print(f"workspace: {workspace}")

exp = Experiment(workspace=workspace, name="filmcheck-fashion")

# 連接已有的 Blob

In [3]:
from azureml.core import Datastore
blob_datastore_name = 'filmcheck'
blob_account_name= ''
blob_account_key = ''
blob_container_name = ''
blob_datastore = Datastore.register_azure_blob_container(workspace=workspace, 
                                                         datastore_name=blob_datastore_name, 
                                                         container_name=blob_container_name, 
                                                         account_name=blob_account_name,
                                                         account_key=blob_account_key)

# 建立運算叢集

In [ ]:
cluster_name = "cpu-cluster-4core"

found = False

cts = workspace.compute_targets
if cluster_name in cts and cts[cluster_name].type == "AmlCompute":
    found = True
    print("Found existing compute target.")
    compute_target = cts[cluster_name]
if not found:
    print("Creating a new compute target...")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size= "STANDARD_DS3_V2",
        max_nodes=1,
    )
    compute_target = ComputeTarget.create(workspace, cluster_name, compute_config)
    compute_target.wait_for_completion(
        show_output=True, min_node_count=None, timeout_in_minutes=10
    )

# 設定執行環境的 config

In [ ]:
run_config = RunConfiguration()
run_config.environment.docker.enabled = True
conda = CondaDependencies()
conda.add_pip_package('azureml-sdk[automl]')
conda.add_pip_package('opencv-python-headless')
run_config.environment.python.conda_dependencies = conda

# 建立 Python 腳本步驟

In [ ]:
datastore = workspace.get_default_datastore()
processed_data = PipelineData('processed_data', datastore=datastore)

script_folder = "./pipeline-python"

prep_step = PythonScriptStep(
    name="go step",
    script_name="go.py",
    arguments=[
        "--output_path", processed_data,
        "--blob_datastore_name", blob_datastore_name,
        "--blob_account_name", blob_account_name,
        "--blob_account_key", blob_account_key,
        "--blob_container_name", blob_container_name,
    ],
    outputs=[processed_data],
    source_directory=script_folder,
    compute_target=compute_target,
    runconfig=run_config,
    allow_reuse=False,
)

pipeline = Pipeline(workspace, steps=[prep_step])
print("Pipeline is built")

# 部署成 pipeline 與儲存 pipeline ID

In [ ]:
published_pipeline = pipeline.publish(name="blob-trigger-pipeline", description="blob-trigger-pipeline", continue_on_step_failure=True)
open('pipeline.id', 'w').write(published_pipeline.id)